In [6]:
# Import the required libraries and dependencies

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
%matplotlib inline

# import SDK libraries from OpenSea / MagicEden if/when available



In [7]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [ ]:
# Set the Free Crypto API Call endpoint URLs for the held cryptocurrency assets

btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"


In [ ]:
# Make an API call to access the current price of BTC

btc_response = requests.get(btc_url).json()

# Display the response data
print(json.dumps(btc_response, indent=4, sort_keys=True))

In [ ]:
# Make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Display the response data
print(json.dumps(eth_response, indent=4, sort_keys=True))

In [ ]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response["data"]["1"]["quotes"]["USD"]["price"]

# Print the current price of BTC
btc_price

In [ ]:
# Navigate the BTC response object to access the current price of ETH
eth_price = eth_response["data"]["1027"]["quotes"]["USD"]["price"]

# Print the current price of ETH
eth_price

In [ ]:
# Set the current amount of shares held in both the stock (SPY) and bond (AGG) portion of the portfolio.
spy_shares = 110
agg_shares = 200

In [ ]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ["SPY", "AGG"]

# Set timeframe to 1Day
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [ ]:
# Use the Alpaca get_bars function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Reorganize the DataFrame
# Separate ticker data
SPY = prices_df[prices_df['symbol']=='SPY'].drop('symbol', axis=1)
AGG = prices_df[prices_df['symbol']=='AGG'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
prices_df = pd.concat([SPY, AGG], axis=1, keys=["SPY", "AGG"])

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

In [ ]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = float(prices_df["AGG"]["close"])

# Print the AGG closing price
agg_close_price

In [ ]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = float(prices_df["SPY"]["close"])

# Print the AGG closing price
spy_close_price

In [ ]:
# Request daily_returns data for a collection from OpenSea API

# os_response = requests.get(<INSERT OS ENDPOINT>).json()



In [ ]:
# Request daily_returns data for a collection from MagicEden API

# me_response = requests.get(<INSERT ME ENDPOINT>).json()



In [ ]:
# Calculate the pct_change of OS and ME dataframes


In [ ]:
# Calculate the covariance, beta, and sharpe ratios of OS and ME dataframes


In [ ]:
# Perform Monte Carlo simulations for OS and ME data

In [42]:
# Set the base url for the NFTPort API
base_nftport_url = "https://api.nftport.xyz/v0"

# Set the headers, including for authentication
token = os.getenv("NFTPORT_API_KEY")
headers = {'Authorization': token}


In [46]:
def get_nfts_for_account(chain, account):
  
    url = base_nftport_url + "/accounts/" + account
    display(url)
    
    display(headers)
    
    response = requests.get(base_nftport_url + "/accounts/" + account + f"?chain={chain}", headers=headers).json()

    # Display the response data
    print(json.dumps(response, indent=4, sort_keys=True))
    
def get_transactions_for_nft(chain, trans_type, contract_address, token_id):
    
    url = base_nftport_url + "/transactions/nfts/" + contract_address + "/" + token_id + f"?chain={chain}&type={trans_type}"
    display(url)

In [47]:
# response = get_nfts_for_account("ethereum","0xeA70F5fcf0E51900D13B7992B0996530f036A133")

response = get_transactions_for_nft("ethereum", "sale", "0x0eff88e66eb717f8f7e0701345f1cc119b487f86", "4849")



'https://api.nftport.xyz/v0/transactions/nfts/0x0eff88e66eb717f8f7e0701345f1cc119b487f86/4849?chain=ethereum&type=sale'